In [1]:
!pip install torcheval

In [2]:
import wandb

# Initialize a new wandb run
wandb.init(project='siamese-network-rotation-prediction-COLAB')

wandb: Currently logged in as: dtriantafyllidou to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
from timm.models import create_model

from torcheval.metrics import R2Score
from PIL import Image
import argparse
import os
import random
import numpy as np
import wandb
from torchvision.utils import make_grid
import csv


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#%cd /content/drive/MyDrive/Kaggle
#!kaggle datasets download -d awsaf49/coco-2017-dataset
#!wget http://images.cocodataset.org/zips/train2017.zip


/content/drive/MyDrive/Kaggle
--2025-02-25 15:10:22--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.165.225, 16.15.216.127, 52.216.222.65, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.165.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  51.7MB/s    in 6m 2s   

2025-02-25 15:16:25 (50.9 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]



In [ ]:
#!wget http://images.cocodataset.org/zips/val2017.zip

--2025-02-25 15:16:44--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.209.97, 3.5.25.176, 52.216.56.169, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.209.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  22.5MB/s    in 40s     

2025-02-25 15:17:25 (19.2 MB/s) - ‘val2017.zip’ saved [815585330/815585330]



In [5]:
!ls /content/drive/MyDrive/Kaggle


coco2017  train2017  train2017.zip  val2017  val2017.zip  wandb


In [64]:
!cp /content/drive/MyDrive/Kaggle/train2017.zip ./sample_data/train2017.zip


In [ ]:
!cp /content/drive/MyDrive/Kaggle/train2017.zip ./sample_data/train2017.zip


In [4]:
!ls ./sample_data

anscombe.json		      mnist_test.csv	     train2017	    val2017_angles.csv
california_housing_test.csv   mnist_train_small.csv  train2017.zip  val2017.zip
california_housing_train.csv  README.md		     val2017


In [66]:
import zipfile
with zipfile.ZipFile('./sample_data/train2017.zip', 'r') as zip_ref:
    zip_ref.extractall('./sample_data/train2017/')
print('Extraction complete!')

Extraction complete!


In [67]:
import zipfile
with zipfile.ZipFile('./sample_data/val2017.zip', 'r') as zip_ref:
    zip_ref.extractall('./sample_data/val2017')
print('Extraction complete!')

Extraction complete!


In [7]:
import cv2
import numpy as np
import math
from PIL import Image

def rotate_image(image, angle):
    """
    Rotates an OpenCV 2 / NumPy image about it's centre by the given angle
    (in degrees). The returned image will be large enough to hold the entire
    new image, with a black background
    """

    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result


def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.

    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow

    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )


def crop_around_center(image, width, height, crop):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """
    if not crop:
        return image
    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]

width = 224
height = 224
dsize = (width, height)
def rotate_preserve_size(image, angle, dsize=dsize, crop=True):
    image_height, image_width = image.shape[0:2]
    image_orig = np.copy(image)
    image_rotated = rotate_image(image, angle)
    width, height = largest_rotated_rect(image_width, image_height, math.radians(angle))
    image_rotated_cropped = crop_around_center(image_rotated, width, height, crop=crop)
    final_image = Image.fromarray(image_rotated_cropped)
    #Using the PIL libaray converting the numpy array to comaptible image
    final_image = final_image.resize(size=dsize)
    return final_image



Data Loader

In [32]:
class COCODataset(Dataset):
    """
    Custom Dataset class for the COCO Dataset. There is the option to generate random rotation angles online (on-the-fly);
    This can be done by setting the parameter is_train=True. There is also the option to use a fixed set of angle parameters
    which can be obtained through a csv file; the latter is useful for logging consistent validation/test results.

    Args:
        root (str): Root directory containing the image dataset.
        is_train (bool, optional): If True we generate random rotation angles on-the-fly.
        angles_file (str, optional): Path to the CSV file containing angles for validation/test images. Defaults is None.

    Methods:
        load_angles_from_csv(angles_file): Loads rotation angles from a CSV file.
        __getitem__(index): Retrieves an image and its rotated version along with the rotation angle.
        __len__(): Returns the total number of images in the dataset.
    """

    def __init__(self, root, is_train=True, angles_file=None):
        self.root = root

        self.image_files = sorted(os.listdir(root))
        self.image_files = [x for x in self.image_files if x != ".DS_Store" ]

        self.image_files = [x for x in self.image_files if x != '.ipynb_checkpoints' ]

        self.is_train = is_train

        self.transform = transform
        if not self.is_train and angles_file is not None:
            self.angles = self.load_angles_from_csv(angles_file)
        else:
            self.angles = None

    def load_angles_from_csv(self, angles_file):
        angles_dict = {}
        with open(angles_file, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                angles_dict[row['image_file']] = float(row['angle'])
        return [angles_dict[img_file] for img_file in self.image_files]

    def __getitem__(self, index):
        path = self.image_files[index]
        if os.path.exists(os.path.join(self.root, path)):
            img = Image.open(os.path.join(self.root, path)).convert('RGB')

        if self.is_train:
            degree = random.uniform(0, 359)  # Generate a random rotation degree
        else:
            degree = self.angles[index]

        img_np = np.asarray(img)
        rotated_img_pil = rotate_preserve_size(img_np, degree, ((224, 224)))

        img_tensor = self.transform(img)
        rotated_tensor = self.transform(rotated_img_pil)

        norm_degree = degree
        return img_tensor, rotated_tensor, torch.tensor(norm_degree, dtype=torch.float32), self.image_files[index]

    def __len__(self):
        return len(self.image_files)


In [9]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


Model definition

In [10]:

class SiameseResNet(nn.Module):
    def __init__(self):
        super(SiameseResNet, self).__init__()
        resnet = models.resnet50(pretrained=True)  # You can use other ResNet versions if needed
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  # Remove the last FC layer

        self.fc = nn.Sequential(
            nn.Linear(4096, 359),
        )

        self.fc.apply(initialize_weights)

    def forward_one(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)

        combined_output = torch.cat((output1, output2), dim=1)
        output = self.fc(combined_output)

        print("SHAPE HERE IS SHAPE ")
        print(combined_output.shape)

        output = output.squeeze()

        return output


In [11]:
class SiameseNetwork(nn.Module):
    def __init__(self, backbone='vit_base_patch16_224'):
        super(SiameseNetwork, self).__init__()
        self.encoder = create_model(backbone, pretrained=True, num_classes=0)
        self.fc = nn.Linear(2 * self.encoder.num_features*2, 359)

    def forward(self, img_a, img_b):
        feat_a = self.encoder(img_a)
        feat_b = self.encoder(img_b)
        combined = torch.cat((feat_a, feat_b), dim=1)
        rotation_angle = self.fc(combined)
        return rotation_angle



In [12]:
# Define the ResNet3D model
class ResNet3D(nn.Module):
    def __init__(self, num_classes=4):
        super(ResNet3D, self).__init__()
        self.resnet3d = models.video.r3d_18(pretrained=True)
        self.resnet3d.fc = nn.Linear(self.resnet3d.fc.in_features, 359)

    def forward(self, input1, input2):
        input1 = input1.unsqueeze(2)  # Add a depth dimension to input1
        input2 = input2.unsqueeze(2)  # Add a depth dimension to input2
        x = torch.cat((input1, input2), dim=2)  # Concatenate along the depth dimension
        output = self.resnet3d(x)
        return output


Training

In [13]:


device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
#model = SiameseResNet().to(device)
model = ResNet3D().to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
num_epochs=10
save_freq=1
print_freq=100
batch_size=32
root_dir="./"
num_workers=8
wandb_log=True

In [15]:

def train(model, epochs=10, save_frequency=1, print_freq=100, batch_size=32, root_dir="./", num_workers=1,
          wandb_log=True):
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

    # Paths to training and validation datasets
    train_root = os.path.join( './sample_data/train2017/train2017/')
    val_root = os.path.join('./sample_data/val2017/val2017/')
    angles_file = os.path.join( './sample_data/val2017_angles.csv')

    # Create datasets
    train_dataset = COCODataset(root=train_root)
    val_dataset = COCODataset(root=val_root, is_train=False, angles_file=angles_file)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    # Initialize the device and move the model to the device
    device = torch.device("mps" if torch.has_mps else "cpu")
    r2_metric = R2Score().to(device)
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")


    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_mae = 0.0

        model.train()
        for i, (input1, input2, rotation_angle, _) in enumerate(train_loader):

            input1 = input1.to(torch.float32).to(device, non_blocking=True)
            input2 = input2.to(torch.float32).to(device, non_blocking=True)

            rotation_angle = rotation_angle.type(torch.LongTensor).to(device)

            optimizer.zero_grad()
            output = model(input1, input2)

            loss = F.cross_entropy(output, rotation_angle, reduction="mean")
            predictions = torch.argmax(output, dim=1)
            correct_predictions = (torch.sum((predictions == rotation_angle))).float()
            accuracy = correct_predictions / batch_size

            absolute_errors = torch.abs(predictions - rotation_angle)
            mae = torch.mean(absolute_errors.float())
            running_mae += mae

            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Log the loss to wandb
            if i % print_freq == 0:  # Print loss every x batches
                if wandb_log:
                    wandb.log({
                        "Batch Loss": loss.item(),
                        "Batch MAE": mae,
                    })
                print(
                    f'Epoch {epoch + 1}, Batch {i + 1}, Batch Loss: {loss.item():.6f}, Batch Accuracy: {accuracy:.6f}, Batch MAE: {mae:.6f}')

        train_loss = running_loss / len(train_loader)
        train_mae = running_mae / len(train_loader)

        model.eval()
        running_loss = 0.0
        all_predictions = []
        all_targets = []

        with torch.no_grad():
            for i, (input1, input2, rotation_angle, _) in enumerate(val_loader):

                input1 = input1.to(torch.float32).to(device, non_blocking=True)
                input2 = input2.to(torch.float32).to(device, non_blocking=True)

                rotation_angle = rotation_angle.type(torch.LongTensor).to(device)

                output = model(input1, input2)
                running_loss += F.cross_entropy(output, rotation_angle, reduction="mean")

                predictions = torch.argmax(output, dim=1)
                all_predictions.append(predictions)
                all_targets.append(rotation_angle)

                # Update R2 metric with predictions and targets
                #r2_metric.update(predictions, rotation_angle)

                # Log images and predictions
                if i % print_freq == 0:
                    if wandb_log:
                        images = []
                        for j in range(input1.size(0)):
                            img = wandb.Image(
                                make_grid([input1[j].cpu(), input2[j].cpu()], nrow=2),
                                caption=f"Pred: {predictions[j].item()  :.2f}°, Actual: {rotation_angle[j].item() :.2f}°"
                            )
                            images.append(img)
                        wandb.log({"Validation Images": images})

        val_loss = running_loss / len(val_loader)

        # Concatenate all predictions and targets
        all_predictions = torch.cat(all_predictions).cpu()
        all_targets = torch.cat(all_targets).cpu()

        # Compute metrics
        val_mae = torch.mean(torch.abs(all_predictions.float() - all_targets.float())).item()
        #val_r2 = r2_metric.compute().item()

        print(f'Epoch {epoch + 1}')
        print(f'Training Loss: {train_loss:.4f}, Training MAE: {train_mae:.4f}')
        print(f'Validation Loss: {val_loss:.4f}, Val MAE: {val_mae:.4f}')

        if wandb_log:
            wandb.log({
                "Train Loss": train_loss,
                "Train MAE": train_mae,
                "Val Loss": val_loss,
                "Val MAE": val_mae,
            })

        # Save the model checkpoint
        if (epoch + 1) % save_frequency == 0:
            model_save_path = '/content/drive/MyDrive/Kaggle'+'/resnet3D_checkpoint_epoch_'+str(epoch)+'.pth'
            torch.save(model.state_dict(), model_save_path)
            print(f"Model checkpoint saved to {model_save_path}")




In [16]:
print(device)

cuda


In [17]:
train(model, epochs=num_epochs, save_frequency=save_freq, print_freq=print_freq,
      batch_size=batch_size, root_dir=root_dir, num_workers=num_workers,
      wandb_log=wandb_log)

<ipython-input-15-aa5eccebad16>:19: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")


Epoch 1, Batch 1, Batch Loss: 6.251999, Batch Accuracy: 0.000000, Batch MAE: 91.687500
Epoch 1, Batch 101, Batch Loss: 5.946847, Batch Accuracy: 0.000000, Batch MAE: 109.187500
Epoch 1, Batch 201, Batch Loss: 5.850841, Batch Accuracy: 0.031250, Batch MAE: 93.125000
Epoch 1, Batch 301, Batch Loss: 5.898538, Batch Accuracy: 0.000000, Batch MAE: 122.000000
Epoch 1, Batch 401, Batch Loss: 5.776685, Batch Accuracy: 0.000000, Batch MAE: 96.437500
Epoch 1, Batch 501, Batch Loss: 5.387969, Batch Accuracy: 0.000000, Batch MAE: 117.375000
Epoch 1, Batch 601, Batch Loss: 5.359957, Batch Accuracy: 0.000000, Batch MAE: 115.812500
Epoch 1, Batch 701, Batch Loss: 5.020021, Batch Accuracy: 0.000000, Batch MAE: 109.968750
Epoch 1, Batch 801, Batch Loss: 4.984286, Batch Accuracy: 0.062500, Batch MAE: 100.250000
Epoch 1, Batch 901, Batch Loss: 4.813973, Batch Accuracy: 0.000000, Batch MAE: 103.750000
Epoch 1, Batch 1001, Batch Loss: 4.758113, Batch Accuracy: 0.062500, Batch MAE: 95.593750
Epoch 1, Batch 

In [18]:
train(model, epochs=num_epochs, save_frequency=save_freq, print_freq=print_freq,
      batch_size=batch_size, root_dir=root_dir, num_workers=num_workers,
      wandb_log=wandb_log)

<ipython-input-15-aa5eccebad16>:19: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")


Epoch 1, Batch 1, Batch Loss: 2.224932, Batch Accuracy: 0.218750, Batch MAE: 7.750000
Epoch 1, Batch 101, Batch Loss: 2.648568, Batch Accuracy: 0.062500, Batch MAE: 6.312500
Epoch 1, Batch 201, Batch Loss: 1.944603, Batch Accuracy: 0.531250, Batch MAE: 6.781250
Epoch 1, Batch 301, Batch Loss: 1.995449, Batch Accuracy: 0.312500, Batch MAE: 4.718750
Epoch 1, Batch 401, Batch Loss: 2.239704, Batch Accuracy: 0.250000, Batch MAE: 2.343750
Epoch 1, Batch 501, Batch Loss: 2.180833, Batch Accuracy: 0.156250, Batch MAE: 7.218750
Epoch 1, Batch 601, Batch Loss: 2.271677, Batch Accuracy: 0.281250, Batch MAE: 6.781250
Epoch 1, Batch 701, Batch Loss: 2.277622, Batch Accuracy: 0.187500, Batch MAE: 2.000000
Epoch 1, Batch 801, Batch Loss: 1.957302, Batch Accuracy: 0.343750, Batch MAE: 6.937500
Epoch 1, Batch 901, Batch Loss: 1.721294, Batch Accuracy: 0.218750, Batch MAE: 1.187500
Epoch 1, Batch 1001, Batch Loss: 2.002114, Batch Accuracy: 0.375000, Batch MAE: 6.687500
Epoch 1, Batch 1101, Batch Loss: 

In [19]:
train(model, epochs=num_epochs, save_frequency=save_freq, print_freq=print_freq,
      batch_size=batch_size, root_dir=root_dir, num_workers=num_workers,
      wandb_log=wandb_log)


<ipython-input-15-aa5eccebad16>:19: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")


Epoch 1, Batch 1, Batch Loss: 1.988274, Batch Accuracy: 0.312500, Batch MAE: 1.468750
Epoch 1, Batch 101, Batch Loss: 2.301101, Batch Accuracy: 0.218750, Batch MAE: 2.125000
Epoch 1, Batch 201, Batch Loss: 2.335818, Batch Accuracy: 0.156250, Batch MAE: 2.875000
Epoch 1, Batch 301, Batch Loss: 1.813736, Batch Accuracy: 0.343750, Batch MAE: 1.281250
Epoch 1, Batch 401, Batch Loss: 2.307068, Batch Accuracy: 0.218750, Batch MAE: 2.125000
Epoch 1, Batch 501, Batch Loss: 2.307583, Batch Accuracy: 0.312500, Batch MAE: 1.875000
Epoch 1, Batch 601, Batch Loss: 2.166979, Batch Accuracy: 0.187500, Batch MAE: 2.250000
Epoch 1, Batch 701, Batch Loss: 2.249751, Batch Accuracy: 0.062500, Batch MAE: 11.187500
Epoch 1, Batch 801, Batch Loss: 1.981472, Batch Accuracy: 0.187500, Batch MAE: 1.625000
Epoch 1, Batch 901, Batch Loss: 2.151706, Batch Accuracy: 0.375000, Batch MAE: 2.031250
Epoch 1, Batch 1001, Batch Loss: 1.744212, Batch Accuracy: 0.375000, Batch MAE: 7.343750
Epoch 1, Batch 1101, Batch Loss:

In [20]:
!wget http://images.cocodataset.org/zips/test2017.zip

--2025-02-25 22:23:07--  http://images.cocodataset.org/zips/test2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.192, 3.5.17.112, 3.5.25.210, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.192|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6646970404 (6.2G) [application/zip]
Saving to: ‘test2017.zip’

test2017.zip        100%[===================>]   6.19G  17.3MB/s    in 6m 35s  

2025-02-25 22:29:42 (16.0 MB/s) - ‘test2017.zip’ saved [6646970404/6646970404]



In [22]:
import zipfile
with zipfile.ZipFile('./test2017.zip', 'r') as zip_ref:
    zip_ref.extractall('./sample_data/test2017/')
print('Extraction complete!')

Extraction complete!


In [38]:
def test(model: nn.Module, print_freq: int = 100, batch_size: int = 32, root_dir: str = "./", num_workers: int = 1,
         wandb_log: bool = True) -> None:

    test_root = os.path.join(root_dir, './sample_data/test2017/test2017/')
    angles_file = os.path.join(root_dir, './sample_data/test2017_angles.csv')

    # Create datasets
    test_dataset = COCODataset(root=test_root, is_train=False, angles_file=angles_file)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

    # Initialize the device and move the model to the device
    device = torch.device(
        "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    model.eval()
    running_loss = 0.0
    all_predictions = []
    all_targets = []

    # r2_metric = R2Score().to(device)

    with torch.no_grad():
        for i, (input1, input2, rotation_angle, _) in enumerate(tqdm(test_loader, desc="Progress")):
            input1 = input1.to(torch.float32).to(device, non_blocking=True)
            input2 = input2.to(torch.float32).to(device, non_blocking=True)

            rotation_angle = rotation_angle.type(torch.LongTensor).to(device)

            output = model(input1, input2)
            running_loss += F.cross_entropy(output, rotation_angle, reduction="mean")

            predictions = torch.argmax(output, dim=1)
            all_predictions.append(predictions)
            all_targets.append(rotation_angle)

            # r2_metric.update(predictions, rotation_angle)

            # Log images and predictions
            if i % print_freq == 0:
                if wandb_log:
                    images = []
                    for j in range(input1.size(0)):
                        img = wandb.Image(
                            make_grid([input1[j].cpu(), input2[j].cpu()], nrow=2),
                            caption=f"Pred: {predictions[j].item()  :.2f}°, Actual: {rotation_angle[j].item() :.2f}°"
                        )
                        images.append(img)
                    wandb.log({"Test Images": images})

    test_loss = running_loss / len(test_loader)

    all_predictions = torch.cat(all_predictions).cpu()
    all_targets = torch.cat(all_targets).cpu()

    test_mae = torch.mean(torch.abs(all_predictions.float() - all_targets.float())).item()
    # test_r2 = r2_metric.compute().item()
    print(f'Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}')



In [39]:
load_model_path = '/content/drive/MyDrive/Kaggle'+'/resnet3D_checkpoint_epoch_'+str(7)+'.pth'

model_save_path = os.path.join(load_model_path)  # Update with the correct path

if os.path.exists(model_save_path):
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    print(f"Loaded pretrained weights from {model_save_path}")
else:
    raise FileNotFoundError(f"No pretrained weights found at {args.model_save_path}")


Loaded pretrained weights from /content/drive/MyDrive/Kaggle/resnet3D_checkpoint_epoch_7.pth


<ipython-input-39-59d74d9c6530>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path, map_location=device))


In [40]:
model = model.to(device)

In [41]:
test(model, print_freq=print_freq, batch_size=32, root_dir="./",
             wandb_log=True)

cuda


Progress: 100%|██████████| 1271/1271 [09:30<00:00,  2.23it/s]

Test Loss: 1.7913, Test MAE: 2.0774
